<a href="https://colab.research.google.com/github/Malwina9912/Uczenie-maszynowe-/blob/main/1_Uczenie_maszynowe_case_2_Malwina_Pawlicka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Ładowanie danych z pliku ratings.csv
ratings_url = 'https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv'
ratings = pd.read_csv(ratings_url)



In [2]:
# Ładowanie danych z pliku movies.csv
movies_url = 'https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv'
movies = pd.read_csv(movies_url)

In [3]:
sci_fi_count = movies[movies['genres'].str.contains('Sci-Fi')].shape[0]
print("Liczba filmów Sci-Fi:", sci_fi_count)



Liczba filmów Sci-Fi: 980


In [4]:
# Łączenie danych z ocenami komedii
comedy_ratings = ratings[ratings['movieId'].isin(movies[movies['genres'].str.contains('Comedy')]['movieId'])]



In [5]:
# Obliczanie rozkładu ocen
comedy_ratings_distribution = comedy_ratings['rating'].value_counts().sort_index()
print("Rozkład ocen komedii:\n", comedy_ratings_distribution)


Rozkład ocen komedii:
 rating
0.5     632
1.0    1317
1.5     895
2.0    3405
2.5    2530
3.0    8306
3.5    5086
4.0    9659
4.5    2794
5.0    4429
Name: count, dtype: int64


In [6]:
# Filmy akcji
action_movies = movies[movies['genres'].str.contains('Action')]

In [7]:
# Oceny filmów akcji
action_ratings = ratings[ratings['movieId'].isin(action_movies['movieId'])]

In [8]:
# Średnia ocen wszystkich filmów akcji
action_mean_rating = action_ratings['rating'].mean()
print("Średnia ocen filmów akcji:", action_mean_rating)

Średnia ocen filmów akcji: 3.447984331646809


In [9]:
# 3 najwyżej oceniane filmy akcji
top_3_action_movies = action_ratings.groupby('movieId').agg({'rating': 'mean'}).nlargest(3, 'rating')
print("3 filmy najwyżej oceniane w kategorii akcji:\n", top_3_action_movies)

3 filmy najwyżej oceniane w kategorii akcji:
          rating
movieId        
876         5.0
1631        5.0
2196        5.0


In [45]:
# Indeksy filmów
top_3_action_movie_ids = top_3_action_movies.index

# Tytuły filmów
top_3_action_movie_titles = movies[movies['movieId'].isin(top_3_action_movie_ids)]['title']

print("Tytuły 3 najwyżej ocenianych filmów w kategorii akcji:")
print(top_3_action_movie_titles)

Tytuły 3 najwyżej ocenianych filmów w kategorii akcji:
666     Supercop 2 (Project S) (Chao ji ji hua) (1993)
1228                            Assignment, The (1997)
1647                                  Knock Off (1998)
Name: title, dtype: object


In [46]:
# Średnia ocen wszystkich filmów akcji
action_movies = movies[movies['genres'].str.contains('Action')]
action_ratings = ratings[ratings['movieId'].isin(action_movies['movieId'])]
average_action_rating = action_ratings['rating'].mean()
print("Średnia ocen wszystkich filmów akcji:", average_action_rating)

# 3 filmy najwyżej oceniane w kategorii akcji
top_3_action_movies


Średnia ocen wszystkich filmów akcji: 3.447984331646809


,rating
movieId,
876,5.0
1631,5.0
2196,5.0


In [10]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162990 sha256=4dc4c0b1f5bf910e657262d638b5a8419ceff81b52e31617ba3d6c9947692cb8
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [11]:
from surprise import Dataset, Reader, SVD, KNNWithMeans
from surprise.model_selection import cross_validate


In [12]:

# Załaduj dane z biblioteki Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [13]:
# Zdefiniuj algorytmy
svd = SVD()
knn_means = KNNWithMeans()

In [14]:
# Przeprowadź walidację krzyżową dla algorytmu SVD
svd_cv_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8752  0.8770  0.8683  0.8681  0.8762  0.8730  0.0039  
MAE (testset)     0.6711  0.6740  0.6706  0.6649  0.6722  0.6705  0.0031  
Fit time          1.64    1.64    4.53    2.87    2.37    2.61    1.07    
Test time         0.22    0.13    0.34    0.26    0.11    0.21    0.09    


In [15]:
# Przeprowadź walidację krzyżową dla algorytmu kNN with Means
knn_means_cv_results = cross_validate(knn_means, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8892  0.9008  0.8988  0.9028  0.8939  0.8971  0.0049  
MAE (testset)     0.6813  0.6871  0.6851  0.6891  0.6851  0.6855  0.0026  
Fit time          0.14    0.23    0.30    0.33    0.22    0.25    0.07    
Test time         2.49    2.58    4.03    2.79    2.16    2.81    0.64    


In [16]:
from surprise.model_selection import GridSearchCV

In [17]:
# Definiujesz siatkę parametrów do przeszukania
param_grid = {'k': [2, 3, 4, 5, 6]}

In [18]:
# Przeszukujesz siatkę parametrów za pomocą metody GridSearchCV
grid_search = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=5)
grid_search.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [19]:
# Wypisujesz najlepsze parametry oraz wyniki
print("Najlepsze parametry:", grid_search.best_params)
print("Najlepszy wynik RMSE:", grid_search.best_score['rmse'])
print("Najlepszy wynik MAE:", grid_search.best_score['mae'])

Najlepsze parametry: {'rmse': {'k': 6}, 'mae': {'k': 6}}
Najlepszy wynik RMSE: 0.9216333695765633
Najlepszy wynik MAE: 0.7087308111468686


In [20]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import train_test_split

In [21]:
# Utwórz obiekt Dataset i Reader
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [22]:
from surprise.model_selection import train_test_split


In [23]:
# Podziel zestaw danych na zestaw treningowy i testowy
trainset, _ = train_test_split(data, test_size=0.2, random_state=42)


In [24]:
# Utwórz instancję algorytmu kNN z wykorzystaniem średnich ocen
algo = KNNWithMeans(k=5, sim_options={'name': 'cosine', 'user_based': False})

In [25]:
# Dopasuj model do danych treningowych
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [26]:
# Display unique movie titles in the dataset
unique_titles = movies['title'].unique()
print(unique_titles)


['Toy Story (1995)' 'Jumanji (1995)' 'Grumpier Old Men (1995)' ...
 'Flint (2017)' 'Bungo Stray Dogs: Dead Apple (2018)'
 'Andrew Dice Clay: Dice Rules (1991)']


In [27]:
# Get movie IDs for "Jumanji" and "Flint"
jumanji_id = movies.loc[movies['title'] == 'Jumanji (1995)', 'movieId'].values[0]
flint_id = movies.loc[movies['title'] == 'Flint (2017)', 'movieId'].values[0]


In [28]:
print(movies[movies['title'] == 'Flint (2017)'])


      movieId         title genres
9739   193585  Flint (2017)  Drama


In [29]:
print(movies[movies['title'] == 'Flint (2017)'])


      movieId         title genres
9739   193585  Flint (2017)  Drama


In [30]:
flint_id = 193585  # Ustawiamy poprawny identyfikator filmu "Flint (2017)"

In [31]:
# Sprawdzenie, czy "Flint (2017)" jest obecny w zbiorze treningowym
if flint_id in trainset.all_items():
    # Znajdź filmy podobne do "Flint"
    flint_neighbors = algo.get_neighbors(flint_id, k=5)


In [32]:
# Sprawdzenie, czy "Flint (2017)" jest obecny w zbiorze treningowym
if flint_id in trainset.all_items():
    # Znajdź filmy podobne do "Flint"
    flint_neighbors = algo.get_neighbors(flint_id, k=5)



In [34]:
# Sprawdzenie, czy "Flint (2017)" jest obecny w zbiorze treningowym
if flint_id in trainset.all_items():
    # Znajdź filmy podobne do "Flint"
    flint_neighbors = algo.get_neighbors(flint_id, k=5)

    # Wyświetl rekomendacje
    print("Rekomendacje po obejrzeniu filmu Flint:")
    for movie_id in flint_neighbors:
        title = movies[movies['movieId'] == movie_id]['title'].values[0]
        print("-", title)
else:
    print("Film 'Flint (2017)' nie jest obecny w zbiorze treningowym.")



Film 'Flint (2017)' nie jest obecny w zbiorze treningowym.


In [37]:
# Znajdź najczęściej oceniane filmy
popular_movies = ratings.groupby('movieId').size().sort_values(ascending=False)
top_movies = popular_movies.index[:5]

# Wyświetl rekomendacje
print("Rekomendowane filmy:")
for movie_id in top_movies:
    title = movies[movies['movieId'] == movie_id]['title'].values[0]
    print("-", title)

Rekomendowane filmy:
- Forrest Gump (1994)
- Shawshank Redemption, The (1994)
- Pulp Fiction (1994)
- Silence of the Lambs, The (1991)
- Matrix, The (1999)


In [ ]:
# Znajdź filmy podobne do "Jumanji"
jumanji_neighbors = algo.get_neighbors(jumanji_id, k=5)

In [38]:
# Wyświetl rekomendacje
print("Rekomendacje po obejrzeniu filmu Jumanji:")
for movie_id in jumanji_neighbors:
    title = movies[movies['movieId'] == movie_id]['title'].values[0]
    print("-", title)

Rekomendacje po obejrzeniu filmu Jumanji:


NameError: name 'jumanji_neighbors' is not defined

In [ ]:
print("\nRekomendacje po obejrzeniu filmu Flint:")
for movie_id in flint_neighbors:
    title = movies[movies['movieId'] == movie_id]['title'].values[0]
    print("-", title)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [40]:
# Utwórz macierz TF-IDF dla kolumny 'genres'
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

In [41]:
# Oblicz podobieństwo kosinusowe między filmami na podstawie macierzy TF-IDF
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [42]:
# Funkcja do generowania rekomendacji na podstawie podobieństwa kosinusowego
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = movies[movies['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

In [43]:
# Rekomendacje dla filmu "Jumanji (1995)"
print("Rekomendacje po obejrzeniu filmu Jumanji:")
print(get_recommendations("Jumanji (1995)"))


Rekomendacje po obejrzeniu filmu Jumanji:
53             Indian in the Cupboard, The (1995)
109             NeverEnding Story III, The (1994)
767               Escape to Witch Mountain (1975)
1514    Darby O'Gill and the Little People (1959)
1556                          Return to Oz (1985)
Name: title, dtype: object


In [44]:
# Rekomendacje dla filmu "Flint (2017)"
print("\nRekomendacje po obejrzeniu filmu Flint:")
print(get_recommendations("Flint (2017)"))


Rekomendacje po obejrzeniu filmu Flint:
25                     Othello (1995)
30             Dangerous Minds (1995)
36    Cry, the Beloved Country (1995)
39                 Restoration (1995)
50                     Georgia (1995)
Name: title, dtype: object
